# Big G Express - Data Exploration

## Team: Elden Ring

<img src="https://eldenring.wiki.fextralife.com/file/Elden-Ring/mirel_pastor_of_vow.jpg" alt="PRAISE DOG" style="width:806px;height:600px;"/>

#### PRAISE THE DOG!

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.feature_selection import chi2
from sklearn.impute import SimpleImputer

Load in the files.

In [2]:
faults = pd.read_csv('../data/J1939Faults.csv', low_memory=False, parse_dates=['EventTimeStamp', 'LocationTimeStamp']) #index_col='EventTimeStamp'
service_fault = pd.read_excel('../data/Service Fault Codes_1_0_0_167.xlsx')
vehicle_diagnostic = pd.read_csv('../data/VehicleDiagnosticOnboardData.csv')

/Users/tomoumer/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


Few keyponts from the Q&A with Josh Treet: 
- throw 2011 dates and older out, mistake with an integer overflow that took a few days to correct
- any timeframe being able to predict a derate is great (even just a few hours)
- derates are going to be related to emissions conditions
- coolant level codes (and some others) can often flip between on and off
- if there's a derate and the light continuing to be on, it's the same event (a pulse of it)
- spn + fmi together determine the fault code
- most trucks fairly similar/same (within 4 years)
- Estimated cost about $500 if misspredicted a derate 

## Exploratory Data Analysis

In [3]:
print(faults.shape)
print(service_fault.shape)
print(vehicle_diagnostic.shape)

(1187335, 20)
(7124, 14)
(12821626, 4)


Faults joins to vehicle_diagnostic with RecordID = FaultID

Columns actionDescription and faultValue in the faults are unused. `faults['actionDescription'].isna().sum()`

We also remove 2169 EquipmentID that have more than 5 characters

In [4]:
faults = (
    faults.drop(['actionDescription', 'faultValue'], axis=1)
    [faults['EquipmentID'].str.len() <= 5]
)

There are three service locations that appear in the dataset. The fault signals might be going on and off there. In order to eliminate those counts, we check if the Latitutde and Longitude coordinates of the truck are within 0.01 units (in both Lat and Long directions) next to a service location. The 0.01 represent, roughly, the distance of a mile.

Doing so, we eliminate 131778 events.

In [5]:
for lat, lon in [(36.0666667, -86.4347222), (35.5883333, -86.4438888), (36.1950, -83.174722)]:
    
    faults = faults.loc[~((abs(lat - faults['Latitude']) <= 0.01) &
                          (abs(lon - faults['Longitude']) <= 0.01))]

Also filter out all erroneous years, 2011 or earlier, (394 lines), caused by an integer error

In [6]:
faults = faults.loc[faults['EventTimeStamp'].dt.year > 2011]

Finally, remove the rows where 'active' column is False -> those represent where an indicator was turned off and that is 506690 rows.

In [7]:
faults = faults.loc[faults['active'] == True]

The final result of these filters is we're left with 546674 rows in faults.

In [ ]:
# save the filtered faults to use for ml
faults.to_pickle('../data/faults_filtered.pkl')

## Exploring Faults with a Rolling Window

Combine the spn and fmi columns together in order to get them ready to one hot encode and use in the rolling window.

> note: need to order by event time stamp rolling window requires time to be sorted

In [8]:
# to not accidentally modify the original dataframe
faults_encoded = faults.copy()

# create spn_fmi combos in single column
faults_encoded['spn_fmi'] = ['_'.join(i) for i in zip(faults_encoded['spn'].astype(str), faults_encoded['fmi'].astype(str))]

# one hot encoded
faults_encoded = pd.get_dummies(faults_encoded, columns=['spn_fmi'], prefix='spn_fmi')

faults_encoded = faults_encoded.sort_values(by='EventTimeStamp')

In [9]:
# to obtain the names of the one hot encoded columns since there are so many
spnfmi_cols = [col for col in faults_encoded.columns if 'spn_fmi' in col]
fixed_cols = ['RecordID', 'spn', 'fmi']

In [10]:
# for some reason, the agg function with sum works without grouping by;
# but when added the groupby, it just keeps running without being able to complete

# d1 = dict.fromkeys(fixed_cols, lambda x: x[-1]) #this function gets the last value in group!
# d2 = dict.fromkeys(spnfmi_cols, 'sum')

# d = {**d1, **d2}

# faults_encoded.groupby('EquipmentID')[['EventTimeStamp'] + fixed_cols + spnfmi_cols].rolling(window = '1d', on = "EventTimeStamp").agg(d)

Using the groupby (for each truck) followed by rolling window, we can get what spn_fmi combinations happened in the past 24 hours (or whatever the time amount is) for each truck separately.

In [11]:
faults_rolling = (
    faults_encoded
    .groupby('EquipmentID')[['EventTimeStamp'] + spnfmi_cols]
    .rolling(window = '1d', on = "EventTimeStamp")
    .sum()
)

faults_rolling = faults_rolling.reset_index()

In [12]:
# to bring in back the spn and fmi information - this was an alternative since the agregate made the kernel crash!
faults_rolling = pd.merge(faults_encoded[fixed_cols],
                          faults_rolling,
                          left_index= True,
                          right_on = 'level_1').drop(columns='level_1')

Some events we couldn't find the description for (it wasn't present in faults or service fault diagnostic).

Now onto figuring out which SPN and FMI might be useful for predicting a derate. the logic here is to randomly sample rows and compare the frequency of codes present there, with respect of the frequency of codes present where a derate occured.

In [13]:
sample_codes = (
    faults_rolling
        .sample(5000)
        .drop(columns=['RecordID','EventTimeStamp','EquipmentID','spn', 'fmi'])
        .sum()
)

# 928 rows have derate as current event
derate_codes = (
    faults_rolling
        .loc[faults_rolling['spn'] == 5246]
        .drop(columns=['RecordID','EventTimeStamp','EquipmentID','spn', 'fmi'])
        .sum()
)

code_differences = (derate_codes / derate_codes.sum()) - (sample_codes / sample_codes.sum())

In [14]:
# note 5246 are all derates, with different fmi combinations
code_differences = (
    code_differences
    .to_frame()
    .reset_index()
    .rename(columns={'index': 'spn_fmi', 0:'rel_frequency'})
)

# We don't care about the frequencies of derates occuring in either case (because derates are what we want to predict)
code_differences = code_differences.loc[~code_differences['spn_fmi'].str.contains('5246')]

Extract just the highest and lowest 15 values below.

In [15]:
top_code_frequencies = code_differences.sort_values(by='rel_frequency', ascending=False).head(15)
bottom_code_frequencies = code_differences.sort_values(by='rel_frequency', ascending=True).head(15)

Additional modifications below in order to be clearer and able to merge it to the service fault table

In [16]:
# separate spn and fmi in two columns again, drop the combined one afterwards
top_code_frequencies[['spn', 'fmi']] = top_code_frequencies['spn_fmi'].str.split('_', expand=True).drop(columns=[0,1]).rename(columns={2:'spn', 3:'fmi'})
top_code_frequencies = top_code_frequencies.drop(columns='spn_fmi')

# convert back to int (from str)
top_code_frequencies['spn'] = top_code_frequencies['spn'].astype(int)
top_code_frequencies['fmi'] = top_code_frequencies['fmi'].astype(int)

#same as above, but for bottom 15
bottom_code_frequencies[['spn', 'fmi']] = bottom_code_frequencies['spn_fmi'].str.split('_', expand=True).drop(columns=[0,1]).rename(columns={2:'spn', 3:'fmi'})
bottom_code_frequencies = bottom_code_frequencies.drop(columns='spn_fmi')

bottom_code_frequencies['spn'] = bottom_code_frequencies['spn'].astype(int)
bottom_code_frequencies['fmi'] = bottom_code_frequencies['fmi'].astype(int)

In [17]:
pd.options.display.max_colwidth = 200 #default is 50
pd.merge(top_code_frequencies, service_fault, left_on=['spn', 'fmi'], right_on=['SPN', 'J1939 FMI'], how='left')[['spn', 'fmi', 'rel_frequency', 'Cummins Description']].drop_duplicates()

,spn,fmi,rel_frequency,Cummins Description
0,1569,31,0.151572,Engine Protection Torque Derate - Condition Exists
1,3362,31,0.033287,Aftertreatment 1 Diesel Exhaust Fluid Dosing Unit Input Lines - Condition Exists
2,4094,18,0.031963,NOx Limits Exceeded Due to Insufficient Reagent Quality - Data Valid But Below Normal Operating Range - Moderately Severe Level
3,1761,19,0.020459,Aftertreatment 1 Diesel Exhaust Fluid Tank Level - Received Network Data in Error
4,1761,9,0.018924,Aftertreatment 1 Diesel Exhaust Fluid Tank Level - Abnormal Update Rate
5,3364,9,0.016805,Aftertreatment Diesel Exhaust Fluid Quality - Abnormal Update Rate
6,5394,17,0.016128,NaN
7,5394,5,0.012212,Aftertreatment 1 Diesel Exhaust Fluid Dosing Valve 1 Circuit - Current Below Normal or Open Circuit
8,6802,31,0.011274,Aftertreatment 1 Diesel Exhaust Fluid Dosing System Frozen - Condition Exists
9,3031,9,0.010882,Aftertreatment 1 Diesel Exhaust Fluid Tank Temperature - Abnormal Update Rate


In [18]:
faults.loc[(faults['spn'] == 3364) & (faults['fmi'] == 9)].head(1)

,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
530883,542760,10637923,2016-07-28 07:50:48,Abnormal Update Rate Catalyst Tank Reagent Quality,04358814*06031374*030816202706*09400153*G1*BDR*,79904672,6X1u13D1500000000,CMMNS,0,3364,9,True,9,1887,105350305,41.113703,-73.390694,2016-07-28 07:51:23


In [19]:
pd.merge(bottom_code_frequencies, service_fault, left_on=['spn', 'fmi'], right_on=['SPN', 'J1939 FMI'], how='left')[['spn', 'fmi', 'rel_frequency', 'Cummins Description']].drop_duplicates()

,spn,fmi,rel_frequency,Cummins Description
0,96,3,-0.139831,Fuel Level (Main Tank) Sensor Circuit - Voltage Above Normal or Shorted to High Source
1,829,3,-0.107641,NaN
2,111,17,-0.106182,Coolant Level - Data Valid But Below Normal Operating Range - Least Severe Level
4,929,9,-0.102582,NaN
5,4096,0,-0.090390,NaN
6,51923,0,-0.064675,NaN
7,596,31,-0.031511,NaN
8,4276,0,-0.021202,NaN
9,0,0,-0.009022,NaN
10,25780,0,-0.007424,NaN


In [20]:
faults.loc[(faults['spn'] == 0) & (faults['fmi'] == 0)].head(1)

,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
436,437,996181,2015-02-21 18:50:17,NaN,unknown,unknown,unknown,unknown,11,0,0,True,127,1620,105416411,38.693888,-84.938055,2015-02-21 18:50:53


Using the codes above that have a positive frequency (more likely to be associated with a derate), their distributions are very close to 0. The ones that have a negative frequency means it is more likely to not be associated with a derate.

In [22]:
faults_rolling.loc[(faults_rolling['spn'] == 5246)][['spn_fmi_1569_31',
	'spn_fmi_3362_31',
    'spn_fmi_4094_18',
    'spn_fmi_1761_19',
    'spn_fmi_1761_9',
    'spn_fmi_3364_9',
    'spn_fmi_5394_17',
    'spn_fmi_5394_5',
    'spn_fmi_6802_31',
    'spn_fmi_3031_9']].describe()

#.to_csv('../data/rolling_trucks.csv')

,spn_fmi_1569_31,spn_fmi_3362_31,spn_fmi_4094_18,spn_fmi_1761_19,spn_fmi_1761_9,spn_fmi_3364_9,spn_fmi_5394_17,spn_fmi_5394_5,spn_fmi_6802_31,spn_fmi_3031_9
count,491.000000,491.000000,491.000000,491.000000,491.000000,491.000000,491.000000,491.000000,491.00000,491.000000
mean,1.004073,0.215886,0.205703,0.134420,0.124236,0.109980,0.103870,0.079430,0.07332,0.071283
std,1.554448,0.749307,0.474286,0.650162,0.625025,0.499103,0.312013,0.325458,0.54466,0.314627
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
50%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
75%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
max,12.000000,7.000000,3.000000,7.000000,7.000000,4.000000,2.000000,2.000000,7.00000,3.000000


Lastly, let's look at the fault codes that tthe trucks have. There are 1042 trucks in the dataset, 330 have partial derate, 28 total and there's 161 with both and 1042 with mome (using the code below).

In [23]:
all_trucks = faults['EquipmentID'].unique()
partial_derate = faults.loc[(faults['spn'] == 1569) & (faults['fmi'] == 31)]['EquipmentID'].unique()
total_derate = faults.loc[faults['spn'] == 5246]['EquipmentID'].unique()

partial_derate_only = partial_derate[np.isin(partial_derate, total_derate, invert=True)]
total_derate_only = total_derate[np.isin(total_derate, partial_derate, invert=True)]
partial_and_total_derate = np.intersect1d(partial_derate, total_derate)
no_derate = all_trucks[np.isin(all_trucks, partial_derate_only, invert=True) | np.isin(all_trucks, total_derate_only, invert=True)]



In [24]:
print(len(partial_derate_only))
print(len(total_derate_only))
print(len(partial_and_total_derate))
print(len(no_derate))

330
28
161
1042


## Creating Predictor Variable

In order to be able to train the models, we need to create a predictor variable. Essentially, look into the "future" of each row and see if a derate happens (let's say within a 6hr window).

To do that, I used a similar approach as above, when performing data exploration. Major difference here is that I sorted the timeseries backwards (in order to look at the future instead of past) and only used spn one hot encoded instead of spn_fmi, because I was looking at full derates that have same spn but different possible fmi.

I also prepped the same for partial derates, but didn't perfom any machin learning on those.

In [25]:
faults_target_derate = faults.copy()
faults_target_75derate = faults.copy()

# the column 'dummy_derate' is so that we can sort derates at the top when the events happen at same time!
# this actually gets additional 48 rows (that are happening at same time)
faults_target_derate['dummy_derate'] = np.where(faults_target_derate['spn'] == 5246, 1, 0)
faults_target_75derate['dummy_derate'] = np.where(faults_target_75derate['spn'] == 1569, 1, 0)

faults_target_derate = pd.get_dummies(faults_target_derate, columns=['spn'], prefix='spn')
faults_target_75derate = pd.get_dummies(faults_target_75derate, columns=['spn'], prefix='spn')

# have to invert the time order here to look into the future!
faults_target_derate = faults_target_derate.sort_values(by=['EquipmentID','EventTimeStamp','dummy_derate'], ascending=[False, False, False])
faults_target_75derate = faults_target_75derate.sort_values(by=['EquipmentID','EventTimeStamp','dummy_derate'], ascending=[False, False, False])

In [26]:
# these are same for both dataframes
var_cols = ['EventTimeStamp'] + [col for col in faults_target_derate.columns if 'spn_' in col]

> NOTE: in the below code we canchange how much into the "future" do we want to look. I attempted various configurations, from 3hr, 6hr, 12 hr, 24 hr, 24 hr and a week. I created a table below to showcase that.

In [27]:
rolling_derate_future = (
    faults_target_derate
    .groupby('EquipmentID')[var_cols]
    .rolling(window = '24h', on = "EventTimeStamp")
    .sum()
    .reset_index()
)

In [28]:
rolling_75derate_future = (
    faults_target_75derate
    .groupby('EquipmentID')[var_cols]
    .rolling(window = '24h', on = "EventTimeStamp")
    .sum()
    .reset_index()
)

In [29]:
rolling_derate_future = pd.merge(faults_target_derate[['RecordID', 'dummy_derate']],
                          rolling_derate_future,
                          left_index= True,
                          right_on = 'level_1').drop(columns='level_1')

rolling_75derate_future = pd.merge(faults_target_75derate[['RecordID', 'dummy_derate']],
                          rolling_75derate_future,
                          left_index= True,
                          right_on = 'level_1').drop(columns='level_1')

In [30]:
# use these next two only if you don't want to have the lines WITH a derate as targets (removes 491 lines), same for partial derates
# rolling_derate_future = rolling_derate_future.loc[rolling_derate_future['dummy_derate'] == 0]
# rolling_75derate_future = rolling_75derate_future.loc[rolling_75derate_future['dummy_derate'] == 0]

rolling_derate_future['target'] = np.where(rolling_derate_future['spn_5246'] > 0, 1, 0)
rolling_75derate_future['target'] = np.where(rolling_75derate_future['spn_1569'] > 0, 1, 0)

In [31]:
# this is just to keep it separate, only the recordID and the two possible targets of interest
y_derate = rolling_derate_future[['RecordID', 'target']]
y_75derate = rolling_75derate_future[['RecordID', 'target']]

| timeframe (in future) | # of derate flags | # of partial derate flags |
| --------------------- | ----------------- | ------------------------- |
| present               | 491               | 5044                      |
| 3h                    | 1084              | 9683                      |
| 6h (base)             | 1389              | 10864                     |
| 6h (excl derate rows) | 898               | 5820                      |
| 12h                   | 1866              | 11800                     |
| 24h                   | 2434              | 14069                     |
| 48h                   | 3231              | 16210                     |
| 1week                 | 6300              | 24794                     |

In [32]:
print(y_derate['target'].sum())
print(y_75derate['target'].sum())

2434
14069


In [ ]:
# save the filtered faults to use for ml the first two are the baselane we as a team started with - 6 hrs
# y_derate.to_pickle('../data/target_derate.pkl')
# y_75derate.to_pickle('../data/target_75derate.pkl')

# y_derate.to_pickle('../data/target_derate3h.pkl')
# y_derate.to_pickle('../data/target_derate12h.pkl')
# y_derate.to_pickle('../data/target_derate24h.pkl')
# y_derate.to_pickle('../data/target_derate48h.pkl')
# y_derate.to_pickle('../data/target_derate1wk.pkl')
# y_derate.to_pickle('../data/target_derate6h_noderaterow.pkl')

### Alternative

Ajay designed a function (below) to return the same output. Compared to the rolling window, this one could be expanded to have some individual functionality instead just aggregate over rows.

In [ ]:
def  GetFilteredSPNbyDays(df_faults, windowTimeframeUnit, day_window):
    df_new = pd.DataFrame(columns = ['RecordID','EquipmentID', 'EventTimeStamp','active', 'spn', 'fmi', 'target'])  #'EventTimeStamp_DateOnly', 'PartialDerate',
    #print(df_new)
    df_new = df_new.astype({'EquipmentID': 'int'})
    dts_evt = ""
    dts_evt_max=""

    hasDerate = False
    # loop through rows of original dataframe and assign new values to columns of new dataframe
    for index, row in df_faults.iterrows():
        #if((row['spn'] == 1569) & (row['fmi'] == 31)):  
        if(row['spn'] == 5246): 
            if(hasDerate == False):
                hasDerate = True
                #df_new.loc[index, 'PartialDerate'] = 1
                df_new.loc[index, 'target'] = 0
                dts_evt = row['EventTimeStamp'] 
                #dts_evt_max = dts_evt - timedelta(days=day_window)
                if(windowTimeframeUnit == "hours"):                   
                    dts_evt_max = dts_evt - timedelta(hours=day_window)
                    #print("hours = dts_evt_max= " + str(dts_evt_max))
                elif(windowTimeframeUnit == "days"):                   
                    dts_evt_max = dts_evt - timedelta(days=day_window)
                    #print("days = dts_evt_max= " + str(dts_evt_max))
                #add cols
                df_new.loc[index, 'RecordID'] = row['RecordID']
                df_new.loc[index, 'EquipmentID'] = row['EquipmentID']
                df_new.loc[index, 'EventTimeStamp'] = row['EventTimeStamp']
                #df_new.loc[index, 'EventTimeStamp_DateOnly'] = row['EventTimeStamp_DateOnly'] 
                df_new.loc[index, 'active'] = row['active']
                df_new.loc[index, 'spn'] = row['spn']
                df_new.loc[index, 'fmi'] = row['fmi']

        else:
            if(dts_evt_max != ""):
 
                if((row['EventTimeStamp'] > dts_evt_max) & (hasDerate)) : #row['EventTimeStamp'] <= dts_evt)
                    #print('EventTimeStamp > dts_evt_max' + str(dts_evt_max) + " ---- " + str(row['EventTimeStamp_DateOnly']))
                    #df_new.loc[index, 'PartialDerate'] = 0 
                    df_new.loc[index, 'target'] = 1
                    df_new.loc[index, 'RecordID'] = row['RecordID']
                    df_new.loc[index, 'EquipmentID'] = row['EquipmentID']
                    df_new.loc[index, 'EventTimeStamp'] = row['EventTimeStamp']
                    #df_new.loc[index, 'EventTimeStamp_DateOnly'] = row['EventTimeStamp_DateOnly'] 
                    df_new.loc[index, 'active'] = row['active']
                    df_new.loc[index, 'spn'] = row['spn']
                    df_new.loc[index, 'fmi'] = row['fmi']
                else:
                    #print('ELSE  ' + str(dts_evt_max)+ " ---- " +  str(row['EventTimeStamp_DateOnly']))
                    hasDerate = False
                    dts_evt = ""
                    dts_evt_max = ""
    #print(df_new) 
    return df_new
    

In [ ]:
df_filtered = pd.DataFrame()
for chunk in pd.read_csv("../data/df_faults.csv", chunksize=5000, parse_dates=['EventTimeStamp']):
    dfW = GetFilteredSPNbyDays(chunk, "hours", 6)
    #print(dfW)
    dfW['active']=dfW['active'].astype(bool) #to avoid futuretype warning
    df_filtered = pd.concat([df_filtered, dfW])

## Vehicle Diagnostic

For vehicle diagnostic:
- Id -  the record Id
- Name – the name of the diagnostic
- Value – the value for that diagnostic
- FaultId – foreign key to the QCJ1939Fault record

Work on the Diagnostic table done by Alison Cordoba

In [33]:
vehicle_diagnostic.head(10)

,Id,Name,Value,FaultId
0,1,IgnStatus,False,1
1,2,EngineOilPressure,0,1
2,3,EngineOilTemperature,96.74375,1
3,4,TurboBoostPressure,0,1
4,5,EngineLoad,11,1
5,6,AcceleratorPedal,0,1
6,7,IntakeManifoldTemperature,78.8,1
7,8,FuelRate,0,1
8,9,FuelLtd,12300.907429328,1
9,10,EngineRpm,0,1


In [34]:
# make a copy of DF to prevent accidental changes
Diagnostics = vehicle_diagnostic

In [35]:
Diagnostics = Diagnostics.pivot(index="FaultId", columns="Name", values="Value").reset_index()

In [36]:
# Remove commas from all 11 columns
for col in Diagnostics.columns[:21]:
    Diagnostics[col] = Diagnostics[col].astype(str).str.replace(',', '')

# Convert all columns to numeric
Diagnostics = Diagnostics.apply(pd.to_numeric, errors='coerce')

Diagnostics

Name,FaultId,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,...,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
0,1,0.0,14.2100,NaN,66.48672,423178.70000,100.4,11.0,0.00,96.74375,...,NaN,NaN,78.8,1023,NaN,NaN,0.00000,3276.75,NaN,0.00
1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
3,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
4,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,16639,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1187330,1248454,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1023,NaN,NaN,NaN,NaN,NaN,NaN
1187331,1248455,100.0,14.5000,NaN,64.62260,423937.90000,185.0,51.0,37.12,211.49370,...,32.0,NaN,98.6,18431,NaN,NaN,65.01096,NaN,73.2,7.83
1187332,1248456,0.0,14.3550,NaN,66.48672,465925.40000,186.8,62.0,41.18,212.84380,...,NaN,NaN,91.4,17407,NaN,NaN,66.57410,NaN,100.0,6.96
1187333,1248457,1.6,14.4275,NaN,67.72946,28606.65625,181.4,0.0,27.26,221.73120,...,NaN,NaN,100.4,1023,NaN,NaN,11.84489,14.10,100.0,1.74


In [37]:
Diagnostics = Diagnostics.drop(columns=['CruiseControlActive', 'IgnStatus', 'ParkingBrake'])

In [38]:
faults_improved = faults.merge(Diagnostics, left_on='RecordID', right_on='FaultId')
faults_improved

,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,...,FuelLtd,FuelRate,FuelTemperature,IntakeManifoldTemperature,LampStatus,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
0,1,990349,2015-02-21 10:47:13,Low (Severity Low) Engine Coolant Level,unknown,unknown,unknown,unknown,0,111,...,12300.907429,0.000000,NaN,78.8,1023,NaN,0.000000,3276.75,NaN,0.00
1,2,990360,2015-02-21 11:34:34,NaN,unknown,unknown,unknown,unknown,11,629,...,NaN,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN
2,4,990370,2015-02-21 11:35:33,Incorrect Data Steering Wheel Angle,unknown,unknown,unknown,unknown,11,1807,...,NaN,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN
3,6,990431,2015-02-21 11:40:22,Low (Severity Low) Engine Coolant Level,04993120*00025921*082113134117*07700053*I0*BBZ*,79466580,6X1u10D1500000000,CMMNS,0,111,...,70349.809964,4.583399,NaN,111.2,1023,NaN,13.602200,3276.75,NaN,6.67
4,7,990439,2015-02-21 11:40:52,Low (Severity Low) Engine Coolant Level,unknown,unknown,unknown,unknown,0,111,...,40961.065437,14.291750,NaN,78.8,1023,NaN,41.534780,3276.75,NaN,20.59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546669,1248448,123899434,2020-03-06 13:12:43,High Voltage (Fuel Level),NaN,NaN,CECU3B-NAMUX4,PACCR,49,96,...,51466.131257,0.620806,NaN,120.2,1279,NaN,0.941766,NaN,100.0,1.16
546670,1248452,123901805,2020-03-06 13:42:48,Low (Severity Medium) Engine Coolant Level,04358814*06030918*051718174436*09401683*G1*BDR*,79904453,6X1u13D1500000000,CMMNS,0,111,...,64491.926797,0.515137,NaN,104.0,2047,NaN,5.932153,NaN,100.0,0.58
546671,1248455,123905139,2020-03-06 14:04:23,Condition Exists Engine Protection Torque Derate,04358814*06099720*030816202706*09400153*G1*BDR*,79932020,6X1u13D1500000000,CMMNS,0,1569,...,58979.184416,7.647805,32.0,98.6,18431,NaN,65.010960,NaN,73.2,7.83
546672,1248456,123905996,2020-03-06 14:13:38,Abnormal Rate of Change Aftertreatment 1 Intake NOx,05317106*05100987*050719120655*09401585*G1*BDR*,79880653,6X1u13D1500000000,CMMNS,0,3216,...,65080.105870,8.995086,NaN,91.4,17407,NaN,66.574100,NaN,100.0,6.96


In [ ]:
# this below takes 8 min to run!!

# cols = ['activeTransitionCount','MCTNumber', 'AcceleratorPedal',
#          'BarometricPressure', 'CruiseControlSetSpeed', 'DistanceLtd',
#          'EngineCoolantTemperature', 'EngineLoad', 'EngineOilPressure', 
#         'EngineOilTemperature', 'EngineRpm', 'EngineTimeLtd', 'FuelLevel', 'FuelLtd', 
#         'FuelRate', 'FuelTemperature', 'IntakeManifoldTemperature', 'LampStatus', 
#         'ServiceDistance', 'Speed', 'SwitchedBatteryVoltage', 'Throttle', 'TurboBoostPressure']


# imputer = SimpleImputer(strategy='mean')


# for column in cols:
#     #print('current ', column)
    
#     equipment_fixed = faults_improved.groupby('EquipmentID')[column].apply(lambda x: imputer.fit_transform(x.values.reshape(-1, 1)))

#     for Id in equipment_fixed.index :
#         #print('id', Id)
        
#         # the flatten will turn an empty array to a single digit and crash the loop
#         # meaning, we still have to impute values for trucks that don't have any single value in a column filled
#         if len(equipment_fixed.loc[Id].flatten() > 0):
#             faults_improved.loc[faults_improved['EquipmentID'] == Id, column] = equipment_fixed.loc[Id].flatten()

In [ ]:
# dropped the columns from saving because that reduces the file size from 200mb to 80mb
# faults_improved.drop(columns=['RecordID', 'ESS_Id', 'EventTimeStamp', 'eventDescription',
#        'ecuSoftwareVersion', 'ecuSerialNumber', 'ecuModel', 'ecuMake',
#        'ecuSource', 'spn', 'fmi', 'active', 'activeTransitionCount',
#        'EquipmentID', 'MCTNumber', 'Latitude', 'Longitude',
#        'LocationTimeStamp']).to_pickle('../data/diagnostics_imputed.pkl')

An alternative to the above code that Michael helped us develop, much faster ... 15.6s instead of 8 min!!!

In [39]:
# Michael's function to deal with the empty lists that were causing trouble above...
imputer = SimpleImputer(strategy='mean') #change this to median

def impute_values(x):
    imputer_results = imputer.fit_transform(x.values.reshape(-1,1))
    
    if len(imputer_results[0]) == 0:
        return np.array([np.nan] * len(x))
    return imputer_results

In [40]:
diagnostics_imputed = faults.merge(Diagnostics, left_on='RecordID', right_on='FaultId')

# this below is needed so that we can simply reassign back to the dataframe
# and that's because if we sort it by EquipmentID, then the grouping and apply is not going to change the order
diagnostics_imputed = diagnostics_imputed.sort_values(by='EquipmentID')

In [41]:
cols = ['activeTransitionCount','MCTNumber', 'AcceleratorPedal',
         'BarometricPressure', 'CruiseControlSetSpeed', 'DistanceLtd',
         'EngineCoolantTemperature', 'EngineLoad', 'EngineOilPressure', 
        'EngineOilTemperature', 'EngineRpm', 'EngineTimeLtd', 'FuelLevel', 'FuelLtd', 
        'FuelRate', 'FuelTemperature', 'IntakeManifoldTemperature', 'LampStatus', 
        'ServiceDistance', 'Speed', 'SwitchedBatteryVoltage', 'Throttle', 'TurboBoostPressure']


for column in cols:
    # double explode!! can't simply assign back because the right side's EquipmentID is not a unique index
    diagnostics_imputed[column] = diagnostics_imputed.groupby('EquipmentID')[column].apply(lambda x: impute_values(x)).explode().explode().array

In [50]:
# save to use for ml models
diagnostics_imputed[['FaultId'] + cols].to_pickle('../data/diagnostics_imputed_median.pkl')

In [42]:
diagnostics_imputed[cols].sort_index()

,activeTransitionCount,MCTNumber,AcceleratorPedal,BarometricPressure,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,...,FuelLtd,FuelRate,FuelTemperature,IntakeManifoldTemperature,LampStatus,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
0,2.0,105354361.0,0.0,14.21,66.48672,423178.7,100.4,11.0,0.0,96.74375,...,12300.907429,0.0,NaN,78.8,1023.0,NaN,0.0,3276.75,NaN,0.0
1,127.0,105354361.0,0.0,14.4275,66.48672,480083.7,183.2,22.0,35.38,207.41565,...,21088.194481,1.664289,NaN,109.4,1279.0,NaN,0.378648,3276.75,NaN,2.03
2,127.0,105336226.0,0.8,14.355,66.48672,526901.1,186.8,35.0,33.35,212.1125,...,83819.678723,5.673113,NaN,107.6,1279.0,NaN,54.044725,3276.75,NaN,6.09
3,1.0,105438630.0,48.0,14.4275,64.6226,470381.4,181.4,30.0,38.28,196.5313,...,70349.809964,4.583399,NaN,111.2,1023.0,NaN,13.6022,3276.75,NaN,6.67
4,2.0,105344243.0,82.8,14.2825,64.6226,278736.7,188.6,80.0,39.44,210.0313,...,40961.065437,14.29175,NaN,78.8,1023.0,NaN,41.53478,3276.75,NaN,20.59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546669,126.0,105355619.0,0.0,14.645,66.48672,391932.6,181.4,11.0,22.62,197.6,...,51466.131257,0.620806,32.0,120.2,1279.0,NaN,0.941766,NaN,100.0,1.16
546670,93.0,105351219.0,0.0,14.355,66.48672,457529.7,181.4,11.0,19.72,207.2188,...,64491.926797,0.515137,32.0,104.0,2047.0,NaN,5.932153,NaN,100.0,0.58
546671,5.0,105354084.0,100.0,14.5,64.6226,423937.9,185.0,51.0,37.12,211.4937,...,58979.184416,7.647805,32.0,98.6,18431.0,NaN,65.01096,NaN,73.2,7.83
546672,1.0,105336308.0,0.0,14.355,66.48672,465925.4,186.8,62.0,41.18,212.8438,...,65080.10587,8.995086,32.0,91.4,17407.0,NaN,66.5741,NaN,100.0,6.96


In [55]:
service_fault.loc[(service_fault['SPN'] == 3226) & (service_fault['J1939 FMI'] == 9)]

,Published in CES 14602,Cummins Fault Code,Revision,PID,SID,MID,J1587 FMI,SPN,J1939 FMI,J2012 Pcode,Lamp Color,Lamp Device,Cummins Description,Algorithm Description
1843,Y,2771,167,Not Mapped,Not Mapped,Not Mapped,9,3226,9,Not Mapped,Amber,Warning,Aftertreatment 1 Outlet NOx Sensor - Abnormal Update Rate,Aftertreatment NOx (Catalyst Exhaust NOx) sensor communication to the ECM via J1939 abnormal
